In [ ]:
import re
import io
import os
import tqdm
import numpy as np
import pandas as pd
import imageio.v2 as imageio
from matplotlib import pyplot as plt

from itertools import product

from scipy.stats import norm as scipy_norm

In [ ]:
def m_numpy(lats, days, theta):
    Z1 = (days - (theta[0] + theta[1]*lats))/(theta[4]/2)
    Z2 = (days - (theta[2] + theta[3]*lats))/(theta[5]/2)
    P1 = scipy_norm.cdf(Z1)
    P2 = 1-scipy_norm.cdf(Z2)
    return np.c_[P1, P2].min(axis=1)
    
path = "/scratch/project_2003104/gtikhono/realtime_birds/export/"
spnames = os.listdir(path)
p = len(spnames)

grid_vec = np.array(list(product(np.arange(60.0, 70.0, 1), np.arange(1.0, 366.0, 10))))

In [ ]:
res = {}

for j in tqdm.tqdm(range(p)):
    sp = spnames[j]
    # print(sp)
    spdir = path+sp+"/"
    fnames = os.listdir(spdir)
    par_dict = {}
    for f in fnames:
        match = re.search(r'_mig\((\d+_\d+)\)', f)
        mig_range = match.group(1)
        days = [int(x) for x in mig_range.split("_")]
        pars = pd.read_csv(spdir + f)
        pars = pars.T
        pars = pars[0]
        par_dict[days[1]] = pars
    par_df = pd.DataFrame.from_dict(par_dict, orient='index')

    M_arr = np.array([m_numpy(grid_vec[:,0], grid_vec[:,1], par_df.iloc[t].values) for t in range(len(par_df))])
    R_list = np.mean(np.square(np.diff(M_arr, axis=0)), axis=1)
    T = len(R_list)
    xs = 7*np.array(range(T))

    res[sp] = R_list

In [ ]:
R = pd.DataFrame(res).T

In [ ]:
R.T.plot(legend=False)

In [ ]:
idx = R.max(axis=1).sort_values(ascending=False).head(10).index
R.loc[idx].T.plot()

In [ ]:
R.max(axis=1).sort_values(ascending=False).hist(bins=20)

In [ ]:
R.max(axis=1).quantile(0.9)

In [ ]:
# R.max(axis=1).quantile(q) for q in [0.75, 0.8, 0.9]

In [ ]:
R.max(axis=1).sort_values(ascending=False).round(3).head(50)

# Idea 1

# Smaller learning rate for all or problematic species?

# If smaller learning rate, then probably need to initialize at previous value?
# Maybe more steps, maybe not

# Idea 2
# Truncate with thresholds: 0.005, 0.01, 0.015,...., 0.0075???
# Change to check if step would break, then stay at last value if true

In [ ]:
R[R.max(axis=1) >= 0.005].index

In [ ]:
R[R.max(axis=1) >= 0.0075].index

In [ ]:
R[R.max(axis=1) >= 0.01].index

In [ ]:
R[R.max(axis=1) >= 0.0125].index

In [ ]:
R[R.max(axis=1) >= 0.015].index

# Timeline

# 2025 results by friday (fitting on all data, including maps + migration parameters)

# updated/final walk-forward on 2024 with idea 1 (and idea 2 on some species??) at 1 week resolution, ideally Friday

# 2025 real experiment, hopefully by Tuesday morning